In [ ]:
from aiida.manage.configuration import load_profile
from aiida.orm import *
from aiida.tools import delete_nodes
from aiida_aurora.calculations import BatteryCyclerExperiment
from aiida_aurora.data import (BatterySampleData, CyclingSpecsData,
                               TomatoSettingsData)
from aiida_aurora.workflows import CyclingSequenceWorkChain

load_profile()


In [ ]:
for ntype in (
        Str,
        List,
        Dict,
        BatterySampleData,
        TomatoSettingsData,
        CyclingSpecsData,
        BatteryCyclerExperiment,
        CyclingSequenceWorkChain,
):
    nodes = QueryBuilder().append(ntype, project=["id"]).all(flat=True)
    delete_nodes(nodes, dry_run=False)

groups = QueryBuilder().append(
    Group,
    filters={
        "label": {
            "like": "aurora/samples/%",
        },
    },
    project=["id"],
).all(flat=True)

for gid in (2, *iter(groups)):
    print(f"Discarding group<{gid}> nodes")
    nodes = [node.pk for node in load_group(gid).nodes]
    delete_nodes(nodes, dry_run=False)

for group in Group.collection.all():
    if group.pk not in {2, 3, 4, 11}:
        print(f"Discarding group<{group.pk}>")
        Group.collection.delete(group.pk)
